In [1]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Загрузка данных
movies = pd.read_csv(r"C:\Users\den19\OneDrive\Рабочий стол\Обучение\Введение в рекомендательные системы\movies.csv")
ratings = pd.read_csv(r"C:\Users\den19\OneDrive\Рабочий стол\Обучение\Введение в рекомендательные системы\ratings.csv")

# Объединение данных
movie_ratings = pd.merge(movies, ratings, on='movieId')

# Фильмы, которые пользователь уже смотрел
watched_movies = movie_ratings[movie_ratings['userId'] == 7]['movieId'].unique()

# Все фильмы в датасете
all_movies = movies['movieId'].unique()

# Фильмы, которые пользователь не смотрел
unwatched_movies = np.setdiff1d(all_movies, watched_movies)

# Создание матрицы пользователь-фильм
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Заполнение пропущенных значений
user_movie_matrix = user_movie_matrix.fillna(0)

# Вычисление косинусного сходства между пользователями
user_similarity = cosine_similarity(user_movie_matrix)

# Преобразование в DataFrame
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

# Получение похожих пользователей
similar_users = user_similarity_df[7].sort_values(ascending=False)[1:11].index

# Фильмы, которые понравились похожим пользователям
recommendations = ratings[
    (ratings['userId'].isin(similar_users)) & 
    (ratings['movieId'].isin(unwatched_movies)) & 
    (ratings['rating'] >= 4.0)
]

# Группировка по movieId и расчет среднего рейтинга
top_recommendations = recommendations.groupby('movieId')['rating'].mean().sort_values(ascending=False).head(3)

# Получение названий топ-3 рекомендованных фильмов
top_3_movies = movies[movies['movieId'].isin(top_recommendations.index)]['title'].tolist()

# Вывод результатов
print("Топ-3 рекомендованных фильма для пользователя с id=7:")
for i, movie in enumerate(top_3_movies, 1):
    print(f"{i}. {movie}")

Топ-3 рекомендованных фильма для пользователя с id=7:
1. Natural Born Killers (1994)
2. Assassination of Jesse James by the Coward Robert Ford, The (2007)
3. WALL·E (2008)
